In [73]:
import numpy as np
import pandas as pd
 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import xgboost as xgb
from lightgbm import LGBMRegressor
import math
from catboost import Pool, CatBoostRegressor,cv

%matplotlib inline

# pd.set_option('display.max_colwidth',1000)
# pd.set_option('display.height',1000)
# pd.set_option('display.max_rows',500)
# pd.set_option('display.max_columns',500)
# pd.set_option('display.width',1000)

In [74]:
# 数据预处理
def clean_data(train_data_X):    
    for j in range(len(np.array(train_data_X.columns))):
        if train_data_X.columns[j] != "ID":
            print("正在处理第%d列数据"%j)
            array = train_data_X[train_data_X.columns[j]]
            num = 0
            len_ = len(train_data_X[train_data_X.columns[j]])
            for i in range(len_):   
                std_ = array.std()
                min_ = array.min()
                max_ = array.max()
                mean_ = array.mean()
                num = array[i]
                std_max =np.float(mean_+3*std_)
                std_min = np.float(mean_-3*std_)
                if (array[0]>std_max) | (array[0]<std_min):
    #                 mean_del = mean_-(array[0]-mean_)/len_
    #                 array[0] = mean_del
                    array[0] = mean_
                if (array[len_-1]>std_max) | (array[len_-1]<std_min):
                    array[len_-1] = mean_
                if i>0 and i<(len_-1):
                    if (array[i]>std_max) | (array[i]<std_min):
                        if (array[i-1]<std_max) and (array[i-1]>std_min) and (array[i+1]<std_max) and (array[i+1]>std_min):
                            array[i]= np.float(array[i-1]+array[i+1])/2
                        elif ((array[i-1]<std_max) and (array[i-1]>std_min))| (array[i+1]>std_max) | (array[i+1]<std_min): 
                            array[i] = array[i-1]
                        elif (array[i-1]<std_max) | (array[i-1]>std_min)|((array[i+1]<std_max) and (array[i+1]>std_min)):
                            array[i] = array[i+1]
            train_data_X[train_data_X.columns[j]] = array
        else:
            pass
            
    return train_data_X
def drop_all_outlier(df):
    df.drop_duplicates(df.columns.drop('ID'), keep='first', inplace=True)
    df.drop(df[(df.电压A > 13.33333) | (df.电压A < 8.3333333)].index,inplace=True)
    df.drop(df[(df.电压B > 13.33333) | (df.电压B < 8.3333333)].index,inplace=True)
    df.drop(df[(df.电压C > 13.33333) | (df.电压C < 8.3333333)].index,inplace=True)
    df.drop(df[(df.现场温度 > 30) | (df.现场温度 < -30)].index,inplace=True)
    df.drop(df[(df.转换效率A > 10)].index,inplace=True)
    df.drop(df[(df.转换效率B > 10)].index,inplace=True)
    df.drop(df[(df.转换效率C > 10)].index,inplace=True)
    df.drop(df[(df.风向 > 36)].index,inplace=True)
    df.drop(df[(df.风速 > 20)].index,inplace=True)
    return df
# 生成数据
def generate_train_data(train_data, test_data, poly=False, select=False):
    y = train_data['发电量']
    X = train_data.drop(['发电量','ID'], axis=1)
    sub_data = test_data.drop(['ID'], axis=1)
    
    polynm = None
    if poly:
        from sklearn.preprocessing import PolynomialFeatures
        polynm = PolynomialFeatures(degree=2, interaction_only=True)
        X = polynm.fit_transform(X)
        sub_data = polynm.transform(sub_data)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)
    
    sm = None
    if select:
        from sklearn.feature_selection import SelectFromModel
        sm = SelectFromModel(GradientBoostingRegressor(random_state=2))
        X_train = sm.fit_transform(X_train, y_train)
        X_test = sm.transform(X_test)
        sub_data = sm.transform(sub_data)
        
    return X_train, X_test, y_train, y_test, sub_data, sm, polynm

def cal_score(mse):
    if isinstance(mse, float):
        return 1 / (1 + math.sqrt(mse))
    else:
        return np.divide(1, 1 + np.sqrt(mse))
#  定义交叉验证函数  
def cross_validation_test(models, train_X_data, train_y_data, cv=10):
    model_name, mse_avg, score_avg = [], [], []
    for i, model in enumerate(models):
        print(i + 1,'- Model:', str(model).split('(')[0])
        model_name.append(str(i + 1) + '.' + str(model).split('(')[0])
#         cv = ShuffleSplit(n_splits=10, test_size=.1, random_state=0)
        nmse = cross_val_score(model, train_X_data[i], train_y_data[i], cv=cv, scoring='neg_mean_squared_error')
        avg_mse = np.average(-nmse)
        scores = cal_score(-nmse)
        avg_score = np.average(scores)
        mse_avg.append(avg_mse)
        score_avg.append(avg_score)
        print('MSE:', -nmse)
        print('Score:', scores)
        print('Average XGB - MSE:', avg_mse, ' - Score:', avg_score, '\n')
    res = pd.DataFrame()
    res['Model'] = model_name
    res['Avg MSE'] = mse_avg
    res['Avg Score'] = score_avg
    return res

# def add_newid(df):
#     ID = df["ID"]
#     df["new_id"]=(np.mod(ID,205))
#     return df
def add_avg(df):
    array = np.array(df["平均功率"])
    newarray=[]
    num = 0
    for i in np.arange(len(array)):
        for j in np.arange(10):
            sum_1=0
            for m in np.arange(j):
                sum_1 += array[m]
            if i<10:
                num = sum_1/(j+1)
            else:
                num = (array[i-1]+array[i-2]+array[i-3]+array[i-4]+array[i-5]+array[i-6]+array[i-7]+array[i-8]+array[i-9])/9
        newarray.append(num)
    df["平均功率history"] = newarray
    return df
def add_avgs(df,str1,str2):
    array = np.array(df[str1])
    newarray=[]
    num = 0
    for i in np.arange(len(array)):
        for j in np.arange(10):
            sum_1=0
            for m in np.arange(j):
                sum_1 += array[m]
            if i<10:
                num = sum_1/(j+1)
            else:
                num = (array[i-1]+array[i-2]+array[i-3]+array[i-4]+array[i-5]+array[i-6]+array[i-7]+array[i-8]+array[i-9])/9
        newarray.append(num)
    df[str2] = newarray
    return df


### 读取数据

In [75]:
train_data = pd.read_csv('../data/public.train.csv')
test_data = pd.read_csv('../data/public.test.csv')

In [76]:
# 创造 电流avg
train_data["电流avg"]=(train_data["电流A"]+train_data["电流B"]+train_data["电流C"])/3
test_data["电流avg"]=(test_data["电流A"]+test_data["电流B"]+test_data["电流C"])/3
# 功率ABC 平均功率
train_data["平均功率"]=train_data["平均功率"]/500
test_data["平均功率"]=test_data["平均功率"]/500
train_data["功率A"]=train_data["功率A"]/500
train_data["功率B"]=train_data["功率B"]/500
train_data["功率C"]=train_data["功率C"]/500
test_data["功率A"]=test_data["功率A"]/500
test_data["功率B"]=test_data["功率B"]/500
test_data["功率C"]=test_data["功率C"]/500
# 光照强度
train_data["光照强度"]=train_data["光照强度"]/75
test_data["光照强度"]=test_data["光照强度"]/75
# 电压ABC
train_data["电压A"]=train_data["电压A"]/60
test_data["电压A"]=test_data["电压A"]/60
train_data["电压B"]=train_data["电压B"]/60
test_data["电压B"]=test_data["电压B"]/60
train_data["电压C"]=train_data["电压C"]/60
test_data["电压C"]=test_data["电压C"]/60
# 转换效率+ABC
train_data["转换效率"]=train_data["转换效率"]/10
test_data["转换效率"]=test_data["转换效率"]/10
train_data["转换效率A"]=train_data["转换效率A"]/10
test_data["转换效率A"]=test_data["转换效率A"]/10
train_data["转换效率B"]=train_data["转换效率B"]/10
test_data["转换效率B"]=test_data["转换效率B"]/10
train_data["转换效率C"]=train_data["转换效率C"]/10
test_data["转换效率C"]=test_data["转换效率C"]/10
#风向
train_data["风向"]=train_data["风向"]/36
test_data["风向"]=test_data["风向"]/36
#板温
train_data["板温"]=train_data["板温"]/3
test_data["板温"]=test_data["板温"]/3
df_result = pd.DataFrame()
df_result['ID'] = list(test_data['ID'])
special_missing_ID = test_data[test_data[(test_data == 0) | (test_data == 0.)].count(axis=1) > 13]['ID']

In [26]:
# import matplotlib.pyplot as plt
# import numpy as np
# plt_X = np.arange(train_data.shape[0])
# plt.scatter(train_data["平均功率"],train_data["发电量"],marker='.')
# plt.ylabel('Plate temperature')
# plt.xlim(0,12)
# plt.show()

In [27]:
# 强特：光照强度，板温，电流ABC 平均功率，功率ABC
# 光照强度0-12 间隔2
# 板温-9 13
# 电流abc 0-12 间隔2
# 功率 0-12 间隔2

### 异常值处理

In [77]:
cleaned_train_data = train_data.copy()
cleaned_train_data = drop_all_outlier(cleaned_train_data)

cleaned_sub_data = test_data.copy()
cleaned_sub_data = drop_all_outlier(cleaned_sub_data)
cleaned_sub_data_ID = cleaned_sub_data['ID']

# all_data = pd.concat([train_data, test_data], axis=0).sort_values(by='ID').reset_index().drop(['index'], axis=1)
# bad_feature = ['ID', '功率A', '功率B', '功率C', '平均功率', '现场温度', '电压A', '电压B', '电压C', '电流B', '电流C', '转换效率', '转换效率A', '转换效率B', '转换效率C']
# bad_index = all_data[bad_feature][
#     (all_data[bad_feature] > all_data[bad_feature].mean() + 2 * all_data[bad_feature].std()) | 
#     (all_data[bad_feature] < all_data[bad_feature].mean() - 2 * all_data[bad_feature].std())
# ].dropna(how='all').index


all_data  = pd.concat([train_data, test_data], axis=0).sort_values(by='ID').reset_index().drop(['index'], axis=1)
bad_feature = ['ID','功率A', '功率B', '功率C', '平均功率', '现场温度', '电压A', '电压B', '电压C', '电流B', '电流C', '转换效率', '转换效率A', '转换效率B', '转换效率C']
bad_index1 = all_data[bad_feature][
    (all_data[bad_feature] > all_data[bad_feature].mean() + 2 * all_data[bad_feature].std()) | 
    (all_data[bad_feature] < all_data[bad_feature].mean() - 2 * all_data[bad_feature].std())
].dropna(how='all').index
bad_index2 = all_data[
    ((all_data['电压A']<8.333)&(all_data['电压A']!=0))|
    ((all_data['电压B']<8.333)&(all_data['电压B']!=0))|
    ((all_data['电压C']<8.333)&(all_data['电压C']!=0))].index
bad_index = pd.Int64Index(list(bad_index1)+list(bad_index2))
# all_data.loc[np.concatenate([bad_index -1,bad_index,bad_index+1])].sort_values(by='ID', ascending=True)


nn_bad_data = all_data.loc[np.concatenate([bad_index - 1, bad_index, bad_index + 1])].sort_values(by='ID', ascending=True).drop_duplicates()
bad_data = all_data.loc[bad_index].sort_values(by='ID', ascending=True)


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  app.launch_new_instance()


In [78]:
# 上下记录均值替代异常值
for idx, line in bad_data.iterrows():
    ID = line['ID']
    col_index = line[bad_feature][ 
        (line[bad_feature] > all_data[bad_feature].mean() + 2 * all_data[bad_feature].std())| 
        (line[bad_feature] < all_data[bad_feature].mean() - 2 * all_data[bad_feature].std())
    ].index
    index = all_data[all_data['ID'] == ID].index
    
    before_offset = 1
    while (idx + before_offset)in bad_index:
        before_offset += 1

    after_offset = 1
    while (idx + after_offset) in bad_index:
        after_offset += 1
    
    replace_value = (all_data.loc[index - before_offset, col_index].values + all_data.loc[index + after_offset, col_index].values) / 2
    all_data.loc[index, col_index] = replace_value[0]

### 拆分数据

In [79]:
#拆分数据
train_data = all_data.drop(all_data[all_data['ID'].isin(df_result['ID'])].index).reset_index().drop(['index'], axis=1)
test_data = all_data[all_data['ID'].isin(df_result['ID'])].drop(['发电量'], axis=1).reset_index().drop(['index'], axis=1)
len(train_data), len(test_data)
# 去除重复值
train_data = train_data.drop_duplicates(train_data.columns.drop('ID'), keep='first')


In [80]:
# 增加平均功率 old
train_data = add_avg(train_data)
test_data = add_avg(test_data)
cleaned_train_data = add_avg(cleaned_train_data)
cleaned_sub_data = add_avg(cleaned_sub_data)
# add_avgs(df,str1,str2)


In [81]:
# 增加 功率ABC old

# train_data = add_avgs(train_data,"平均功率","平均功率old")
train_data = add_avgs(train_data,"功率A","功率Ahistory")
train_data = add_avgs(train_data,"功率B","功率Bhistory")
train_data = add_avgs(train_data,"功率C","功率Chistory")

# test_data = add_avgs(test_data,"平均功率","平均功率old")
test_data = add_avgs(test_data,"功率A","功率Ahistory")
test_data = add_avgs(test_data,"功率B","功率Bhistory")
test_data = add_avgs(test_data,"功率C","功率Chistory")

# cleaned_train_data = add_avgs(cleaned_train_data,"平均功率","平均功率old")
cleaned_train_data = add_avgs(cleaned_train_data,"功率A","功率Ahistory")
cleaned_train_data = add_avgs(cleaned_train_data,"功率B","功率Bhistory")
cleaned_train_data = add_avgs(cleaned_train_data,"功率C","功率Chistory")

# cleaned_sub_data = add_avgs(cleaned_sub_data,"平均功率","平均功率old")
cleaned_sub_data = add_avgs(cleaned_sub_data,"功率A","功率Ahistory")
cleaned_sub_data = add_avgs(cleaned_sub_data,"功率B","功率Bhistory")
cleaned_sub_data = add_avgs(cleaned_sub_data,"功率C","功率Chistory")

In [56]:
# 增加 光照强度history 板温 电流ABC

# train_data = add_avgs(train_data,"平均功率","平均功率old")
train_data = add_avgs(train_data,"光照强度","光照强度history")
train_data = add_avgs(train_data,"板温","板温history")
train_data = add_avgs(train_data,"电流A","电流Ahistory")
train_data = add_avgs(train_data,"电流B","电流Bhistory")
train_data = add_avgs(train_data,"电流C","电流Chistory")

# test_data = add_avgs(test_data,"平均功率","平均功率old")
test_data = add_avgs(test_data,"光照强度","光照强度history")
test_data = add_avgs(test_data,"板温","板温history")
test_data = add_avgs(test_data,"电流A","电流Ahistory")
test_data = add_avgs(test_data,"电流B","电流Bhistory")
test_data = add_avgs(test_data,"电流C","电流Chistory")

# cleaned_train_data = add_avgs(cleaned_train_data,"平均功率","平均功率old")
cleaned_train_data = add_avgs(cleaned_train_data,"光照强度","光照强度history")
cleaned_train_data = add_avgs(cleaned_train_data,"板温","板温history")
cleaned_train_data = add_avgs(cleaned_train_data,"电流A","电流Ahistory")
cleaned_train_data = add_avgs(cleaned_train_data,"电流B","电流Bhistory")
cleaned_train_data = add_avgs(cleaned_train_data,"电流C","电流Chistory")

# cleaned_sub_data = add_avgs(cleaned_sub_data,"平均功率","平均功率old")
cleaned_sub_data = add_avgs(cleaned_sub_data,"光照强度","光照强度history")
cleaned_sub_data = add_avgs(cleaned_sub_data,"板温","板温history")
cleaned_sub_data = add_avgs(cleaned_sub_data,"电流A","电流Ahistory")
cleaned_sub_data = add_avgs(cleaned_sub_data,"电流B","电流Bhistory")
cleaned_sub_data = add_avgs(cleaned_sub_data,"电流C","电流Chistory")

In [34]:
# import matplotlib.pyplot as plt
# import numpy as np
# plt_X = np.arange(train_data.shape[0])
# plt.scatter(train_data["风向"],train_data["平均功率"],marker='.')
# plt.ylabel('Plate temperature')
# plt.xlim(200,300)
# plt.show()

In [82]:
# 强特：光照强度，板温，,平均电流电流ABC 平均功率，功率ABC
# 光照强度0-12 间隔2
# 板温-9 13
# 电流abc 0-12 间隔2
# 功率 0-12 间隔2
def aparse_process(number):
    if number<2:
        return 0
    elif number<4:
        return 1
    elif number<6:
        return 2
    elif number<8:
        return 3
    elif number<10:
        return 4
    else:
        return 5
# 板温 deal
def plate_temperature_process(number):
    if number<-6:
        return 0
    elif number<-3:
        return 1
    elif number<0:
        return 2
    elif number<3:
        return 3
    elif number<6:
        return 4
    elif number<9:
        return 5
    else:
        return 6
    
    
#-------------------------------------------------------------------------------- 
arr1 = ['平均功率','电流A','电流B','电流C','功率A','功率B','功率C','电流avg']
arr2 = ['平均功率_sparse','电流A_sparse','电流B_sparse','电流C_sparse','功率A_sparse','功率B_sparse','功率C_sparse','电流avg_sparse']
arr_len = len(arr1)
for i in range(arr_len):
    train_data[arr2[i]] = train_data[arr1[i]].apply(aparse_process)
    test_data[arr2[i]] = test_data[arr1[i]].apply(aparse_process)
    cleaned_train_data[arr2[i]] = cleaned_train_data[arr1[i]].apply(aparse_process)
    cleaned_sub_data[arr2[i]] = cleaned_sub_data[arr1[i]].apply(aparse_process)

# train_data['板温_sparse'] = train_data['板温'].apply(plate_temperature_process)
# test_data['板温_sparse'] = test_data['板温'].apply(plate_temperature_process)
# cleaned_train_data['板温_sparse'] = cleaned_train_data['板温'].apply(plate_temperature_process)
# cleaned_sub_data['板温_sparse'] = cleaned_sub_data['板温'].apply(plate_temperature_process)

In [83]:
train_data.columns

Index(['ID', '光照强度', '功率A', '功率B', '功率C', '发电量', '平均功率', '板温', '现场温度', '电压A',
       '电压B', '电压C', '电流A', '电流B', '电流C', '电流avg', '转换效率', '转换效率A', '转换效率B',
       '转换效率C', '风向', '风速', '平均功率history', '功率Ahistory', '功率Bhistory',
       '功率Chistory', '平均功率_sparse', '电流A_sparse', '电流B_sparse', '电流C_sparse',
       '功率A_sparse', '功率B_sparse', '功率C_sparse', '电流avg_sparse'],
      dtype='object')

In [84]:
X_train, X_test, y_train, y_test, sub_data, sm, polynm = generate_train_data(train_data, test_data, poly=True, select=True)

clean_X_train, clean_X_test, clean_y_train, clean_y_test, clean_sub_data, _, _ = generate_train_data(cleaned_train_data, cleaned_sub_data, poly=False, select=False)

clean_X = np.concatenate([clean_X_train, clean_X_test])
clean_y = np.concatenate([clean_y_train, clean_y_test])
clean_X = polynm.transform(clean_X)
clean_X = sm.transform(clean_X)

clean_sub_data = polynm.transform(clean_sub_data)
clean_sub_data = sm.transform(clean_sub_data)

In [61]:
# print(type(X_train))
# print(type(X_test))
# print(type(y_train))
# print(type(y_test))
# print(type(sub_data))
# print(type(sm))
# print(type(polynm))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'sklearn.feature_selection.from_model.SelectFromModel'>
<class 'sklearn.preprocessing.data.PolynomialFeatures'>


In [62]:
# print(type(clean_X_train))
# print(type(clean_X_test))
# print(type(clean_y_train))
# print(type(clean_y_test))
# print(type(clean_sub_data))


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>


## Stacking Model

### Tree Models

In [85]:
all_X_train = np.concatenate([X_train, X_test])
all_y_train = np.concatenate([y_train, y_test])

In [86]:
xgbt1 = xgb.XGBRegressor(silent=1,max_leaf_nodes=255,seed=789,eta=0.03,scoring='neg_mean_squared_error',n_estimators=900, max_depth=5, max_features='sqrt', random_state=777, n_jobs=12)
# xgbt2 = xgb.XGBRegressor(silent=1,max_leaf_nodes=255,seed=1000,eta=0.03,scoring='neg_mean_squared_error',n_estimators=950, max_depth=6, max_features='sqrt', random_state=999, n_jobs=12)
# xgbt3 = xgb.XGBRegressor(silent=1,max_leaf_nodes=255,seed=1515,eta=0.03,scoring='neg_mean_squared_error',n_estimators=1000, max_depth=5, max_features='sqrt', random_state=367, n_jobs=12)
# n_estimators=1000  max_depth=5  'sqrt'  GradientBoostingRegressor 最佳参数 ,learning_rate=0.08
gbdt1 = GradientBoostingRegressor(n_estimators=1060, max_depth=5, max_features='log2', random_state=789,learning_rate=0.08)
# gbdt2 = GradientBoostingRegressor(n_estimators=1300, max_depth=5, max_features='log2', random_state=333,learning_rate=0.08)
# gbdt3 = GradientBoostingRegressor(n_estimators=1500, max_depth=5, max_features='log2', random_state=789,learning_rate=0.08)
# n_estimators=700, max_features='auto', random_state=2, n_jobs=8,max_depth=10
# forest1 = RandomForestRegressor(n_estimators=740, max_features='sqrt', random_state=7, n_jobs=12)
# forest2 = RandomForestRegressor(n_estimators=730, max_features='sqrt', random_state=5, n_jobs=12)
# forest3 = RandomForestRegressor(n_estimators=745, max_features='sqrt', random_state=11, n_jobs=12) 
# n_estimators 850 num_leaves=14,learning_rate=0.1 max_depth=5  seed
cat1 = CatBoostRegressor(iterations=None,
                        learning_rate=0.05,
                        l2_leaf_reg=0.15,
                        border_count=150,
                        max_depth=6,
                        loss_function='RMSE',
                        random_seed=3000,
                        n_estimators=2400,
#                         reg_lambda=2,
                        leaf_estimation_method='Gradient',
                        thread_count=12)
lgb1 = LGBMRegressor(n_estimators=850, max_depth=4, random_state=130, n_jobs=12,num_leaves=14,learning_rate=0.08,seed=666) 
# lgb2 = LGBMRegressor(n_estimators=900, max_depth=4, random_state=521, n_jobs=12,num_leaves=13,learning_rate=0.06,seed=777)
# lgb3 = LGBMRegressor(n_estimators=950, max_depth=4, random_state=777, n_jobs=12,num_leaves=10,learning_rate=0.06,seed=999)

# xgbt1 = xgb.XGBRegressor(silent=1,max_leaf_nodes=255,seed=789,eta=0.03,scoring='neg_mean_squared_error',n_estimators=351, max_depth=5, max_features='sqrt', random_state=777, n_jobs=12)
# xgbt2 = xgb.XGBRegressor(silent=1,max_leaf_nodes=255,seed=1000,eta=0.03,scoring='neg_mean_squared_error',n_estimators=361, max_depth=5, max_features='sqrt', random_state=999, n_jobs=12)
# xgbt3 = xgb.XGBRegressor(silent=1,max_leaf_nodes=255,seed=1515,eta=0.03,scoring='neg_mean_squared_error',n_estimators=371, max_depth=5, max_features='sqrt', random_state=367, n_jobs=12)
# # n_estimators=1000  max_depth=5  'sqrt'  GradientBoostingRegressor 最佳参数 ,learning_rate=0.08
# gbdt1 = GradientBoostingRegressor(n_estimators=1060, max_depth=5, max_features='log2', random_state=789,learning_rate=0.08,n_jobs=-1)
# gbdt2 = GradientBoostingRegressor(n_estimators=1100, max_depth=5, max_features='log2', random_state=789,learning_rate=0.08,n_jobs=-1)
# gbdt3 = GradientBoostingRegressor(n_estimators=1090, max_depth=5, max_features='log2', random_state=789,learning_rate=0.08,n_jobs=-1)
# # n_estimators=700, max_features='auto', random_state=2, n_jobs=8,max_depth=10
# forest1 = RandomForestRegressor(n_estimators=740, max_features='sqrt', random_state=7, n_jobs=12,n_jobs=-1)
# forest2 = RandomForestRegressor(n_estimators=730, max_features='sqrt', random_state=9, n_jobs=12,n_jobs=-1)
# forest3 = RandomForestRegressor(n_estimators=745, max_features='sqrt', random_state=11, n_jobs=12,n_jobs=-1) 
# # n_estimators 850 num_leaves=14,learning_rate=0.1 max_depth=5  seed
# lgb1 = LGBMRegressor(n_estimators=840, max_depth=4, random_state=789, n_jobs=12,num_leaves=14,learning_rate=0.08,seed=666) 
# lgb2 = LGBMRegressor(n_estimators=845, max_depth=4, random_state=798, n_jobs=12,num_leaves=13,learning_rate=0.06,seed=777)
# lgb3 = LGBMRegressor(n_estimators=850, max_depth=4, random_state=777, n_jobs=12,num_leaves=10,learning_rate=0.07,seed=999)

# xgbt1 = xgb.XGBRegressor(n_estimators=950, max_depth=3, max_features='sqrt', random_state=2, n_jobs=8)
# xgbt2 = xgb.XGBRegressor(n_estimators=1000, max_depth=3, max_features='sqrt', random_state=3, n_jobs=8)
# xgbt3 = xgb.XGBRegressor(n_estimators=1100, max_depth=3, max_features='sqrt', random_state=4, n_jobs=8)

# gbdt1 = GradientBoostingRegressor(n_estimators=500, max_depth=3, max_features='sqrt', random_state=2)
# gbdt2 = GradientBoostingRegressor(n_estimators=400, max_depth=3, max_features='sqrt', random_state=3)
# gbdt3 = GradientBoostingRegressor(n_estimators=500, max_depth=4, max_features='log2', random_state=4)

# forest1 = RandomForestRegressor(n_estimators=300, max_features='sqrt', random_state=2, n_jobs=8)
# forest2 = RandomForestRegressor(n_estimators=300, max_features='log2', random_state=3, n_jobs=8)
# forest3 = RandomForestRegressor(n_estimators=600, max_features='sqrt', random_state=4, n_jobs=8) 

# lgb1 = LGBMRegressor(n_estimators=900, max_depth=5, random_state=2, n_jobs=8) 
# lgb2 = LGBMRegressor(n_estimators=850, max_depth=4, random_state=3, n_jobs=8)
# lgb3 = LGBMRegressor(n_estimators=720, max_depth=4, random_state=4, n_jobs=8)

cross_validation_test(
    models=[    
        xgbt1, #xgbt2, xgbt3,
        gbdt1, #gbdt2, gbdt3,
        cat1,
#         forest1, #forest2, forest3,
        lgb1, #lgb2, lgb3
    ],
    train_X_data=[
        all_X_train, all_X_train, all_X_train, all_X_train,
#         all_X_train, all_X_train, all_X_train, all_X_train,
#         all_X_train, all_X_train, all_X_train, all_X_train
    ],
    train_y_data=[
        all_y_train, all_y_train, all_y_train, all_y_train,
#         all_y_train, all_y_train, all_y_train, all_y_train,
#         all_y_train, all_y_train, all_y_train, all_y_train
    ]
)


1 - Model: XGBRegressor
MSE: [0.01185235 0.00822968 0.00920827 0.01532451 0.02177072 0.00936828
 0.01078612 0.16449408 0.00711689 0.01518864]
Score: [0.90182019 0.91682763 0.91244229 0.88984421 0.87142245 0.91175167
 0.90591506 0.71145074 0.92220157 0.89027994]
Average XGB - MSE: 0.027333953641302717  - Score: 0.8833955743061409 

2 - Model: GradientBoostingRegressor
MSE: [0.01222933 0.00988634 0.00987154 0.01899414 0.02027943 0.00854113
 0.00946466 0.16022221 0.00705841 0.01468143]
Score: [0.9004253  0.90956217 0.90962376 0.87887424 0.87534565 0.91540032
 0.91133905 0.71414408 0.92249704 0.89192783]
Average XGB - MSE: 0.027122860104907675  - Score: 0.8829139432739763 

3 - Model: <catboost.core.CatBoostRegressor object at 0x0000019019874CF8>
0:	learn: 6.3724550	total: 32.7ms	remaining: 1m 18s
1:	learn: 6.0605446	total: 65.1ms	remaining: 1m 18s
2:	learn: 5.7618976	total: 98.4ms	remaining: 1m 18s
3:	learn: 5.4808210	total: 132ms	remaining: 1m 18s
4:	learn: 5.2114316	total: 164ms	remaini

143:	learn: 0.1896022	total: 4.92s	remaining: 1m 17s
144:	learn: 0.1890862	total: 4.96s	remaining: 1m 17s
145:	learn: 0.1889340	total: 4.99s	remaining: 1m 17s
146:	learn: 0.1886515	total: 5.02s	remaining: 1m 16s
147:	learn: 0.1884700	total: 5.05s	remaining: 1m 16s
148:	learn: 0.1881728	total: 5.09s	remaining: 1m 16s
149:	learn: 0.1879319	total: 5.12s	remaining: 1m 16s
150:	learn: 0.1860521	total: 5.16s	remaining: 1m 16s
151:	learn: 0.1856923	total: 5.2s	remaining: 1m 16s
152:	learn: 0.1855350	total: 5.23s	remaining: 1m 16s
153:	learn: 0.1854322	total: 5.26s	remaining: 1m 16s
154:	learn: 0.1850599	total: 5.29s	remaining: 1m 16s
155:	learn: 0.1848745	total: 5.33s	remaining: 1m 16s
156:	learn: 0.1845124	total: 5.37s	remaining: 1m 16s
157:	learn: 0.1843328	total: 5.4s	remaining: 1m 16s
158:	learn: 0.1840928	total: 5.43s	remaining: 1m 16s
159:	learn: 0.1837730	total: 5.46s	remaining: 1m 16s
160:	learn: 0.1834465	total: 5.5s	remaining: 1m 16s
161:	learn: 0.1832674	total: 5.53s	remaining: 1m 

304:	learn: 0.1558549	total: 10.3s	remaining: 1m 10s
305:	learn: 0.1557693	total: 10.3s	remaining: 1m 10s
306:	learn: 0.1556686	total: 10.4s	remaining: 1m 10s
307:	learn: 0.1555658	total: 10.4s	remaining: 1m 10s
308:	learn: 0.1555254	total: 10.5s	remaining: 1m 10s
309:	learn: 0.1554497	total: 10.5s	remaining: 1m 10s
310:	learn: 0.1553432	total: 10.5s	remaining: 1m 10s
311:	learn: 0.1552854	total: 10.6s	remaining: 1m 10s
312:	learn: 0.1552570	total: 10.6s	remaining: 1m 10s
313:	learn: 0.1550954	total: 10.6s	remaining: 1m 10s
314:	learn: 0.1550005	total: 10.7s	remaining: 1m 10s
315:	learn: 0.1540566	total: 10.7s	remaining: 1m 10s
316:	learn: 0.1539151	total: 10.7s	remaining: 1m 10s
317:	learn: 0.1528213	total: 10.8s	remaining: 1m 10s
318:	learn: 0.1525982	total: 10.8s	remaining: 1m 10s
319:	learn: 0.1515709	total: 10.8s	remaining: 1m 10s
320:	learn: 0.1515411	total: 10.9s	remaining: 1m 10s
321:	learn: 0.1515191	total: 10.9s	remaining: 1m 10s
322:	learn: 0.1513771	total: 10.9s	remaining: 

462:	learn: 0.1366375	total: 15.6s	remaining: 1m 5s
463:	learn: 0.1365966	total: 15.7s	remaining: 1m 5s
464:	learn: 0.1365817	total: 15.7s	remaining: 1m 5s
465:	learn: 0.1365419	total: 15.8s	remaining: 1m 5s
466:	learn: 0.1364921	total: 15.8s	remaining: 1m 5s
467:	learn: 0.1364114	total: 15.8s	remaining: 1m 5s
468:	learn: 0.1363692	total: 15.9s	remaining: 1m 5s
469:	learn: 0.1363419	total: 15.9s	remaining: 1m 5s
470:	learn: 0.1363275	total: 15.9s	remaining: 1m 5s
471:	learn: 0.1363105	total: 16s	remaining: 1m 5s
472:	learn: 0.1362399	total: 16s	remaining: 1m 5s
473:	learn: 0.1362398	total: 16s	remaining: 1m 5s
474:	learn: 0.1360897	total: 16s	remaining: 1m 4s
475:	learn: 0.1360660	total: 16.1s	remaining: 1m 4s
476:	learn: 0.1360631	total: 16.1s	remaining: 1m 4s
477:	learn: 0.1360022	total: 16.1s	remaining: 1m 4s
478:	learn: 0.1359901	total: 16.2s	remaining: 1m 4s
479:	learn: 0.1359761	total: 16.2s	remaining: 1m 4s
480:	learn: 0.1359392	total: 16.2s	remaining: 1m 4s
481:	learn: 0.135921

627:	learn: 0.1280473	total: 21.1s	remaining: 59.5s
628:	learn: 0.1279895	total: 21.1s	remaining: 59.5s
629:	learn: 0.1278962	total: 21.2s	remaining: 59.4s
630:	learn: 0.1278876	total: 21.2s	remaining: 59.4s
631:	learn: 0.1278801	total: 21.2s	remaining: 59.4s
632:	learn: 0.1278502	total: 21.3s	remaining: 59.3s
633:	learn: 0.1277979	total: 21.3s	remaining: 59.3s
634:	learn: 0.1277510	total: 21.3s	remaining: 59.3s
635:	learn: 0.1277481	total: 21.4s	remaining: 59.3s
636:	learn: 0.1276351	total: 21.4s	remaining: 59.2s
637:	learn: 0.1276204	total: 21.4s	remaining: 59.2s
638:	learn: 0.1275811	total: 21.5s	remaining: 59.2s
639:	learn: 0.1275805	total: 21.5s	remaining: 59.1s
640:	learn: 0.1275803	total: 21.5s	remaining: 59.1s
641:	learn: 0.1275798	total: 21.6s	remaining: 59s
642:	learn: 0.1272683	total: 21.6s	remaining: 59s
643:	learn: 0.1272286	total: 21.6s	remaining: 59s
644:	learn: 0.1267246	total: 21.7s	remaining: 58.9s
645:	learn: 0.1266760	total: 21.7s	remaining: 58.9s
646:	learn: 0.1265

...

1852:	learn: 0.0950015	total: 1m 4s	remaining: 19.1s
1853:	learn: 0.0949986	total: 1m 4s	remaining: 19.1s
1854:	learn: 0.0949986	total: 1m 4s	remaining: 19s
1855:	learn: 0.0949894	total: 1m 4s	remaining: 19s
1856:	learn: 0.0949864	total: 1m 4s	remaining: 19s
1857:	learn: 0.0949506	total: 1m 4s	remaining: 18.9s
1858:	learn: 0.0949504	total: 1m 4s	remaining: 18.9s
1859:	learn: 0.0949125	total: 1m 5s	remaining: 18.9s
1860:	learn: 0.0949058	total: 1m 5s	remaining: 18.8s
1861:	learn: 0.0948955	total: 1m 5s	remaining: 18.8s
1862:	learn: 0.0948583	total: 1m 5s	remaining: 18.8s
1863:	learn: 0.0948469	total: 1m 5s	remaining: 18.7s
1864:	learn: 0.0948423	total: 1m 5s	remaining: 18.7s
1865:	learn: 0.0948423	total: 1m 5s	remaining: 18.7s
1866:	learn: 0.0948349	total: 1m 5s	remaining: 18.6s
1867:	learn: 0.0947702	total: 1m 5s	remaining: 18.6s
1868:	learn: 0.0947683	total: 1m 5s	remaining: 18.6s
1869:	learn: 0.0947656	total: 1m 5s	remaining: 18.5s
1870:	learn: 0.0947644	total: 1m 5s	remaining: 18.5s

2010:	learn: 0.0933441	total: 1m 10s	remaining: 13.5s
2011:	learn: 0.0933275	total: 1m 10s	remaining: 13.5s
2012:	learn: 0.0933183	total: 1m 10s	remaining: 13.5s
2013:	learn: 0.0933134	total: 1m 10s	remaining: 13.4s
2014:	learn: 0.0933053	total: 1m 10s	remaining: 13.4s
2015:	learn: 0.0933032	total: 1m 10s	remaining: 13.4s
2016:	learn: 0.0932575	total: 1m 10s	remaining: 13.3s
2017:	learn: 0.0932525	total: 1m 10s	remaining: 13.3s
2018:	learn: 0.0932505	total: 1m 10s	remaining: 13.3s
2019:	learn: 0.0932283	total: 1m 10s	remaining: 13.2s
2020:	learn: 0.0932243	total: 1m 10s	remaining: 13.2s
2021:	learn: 0.0932240	total: 1m 10s	remaining: 13.2s
2022:	learn: 0.0932173	total: 1m 10s	remaining: 13.1s
2023:	learn: 0.0932168	total: 1m 10s	remaining: 13.1s
2024:	learn: 0.0932145	total: 1m 10s	remaining: 13.1s
2025:	learn: 0.0932087	total: 1m 10s	remaining: 13s
2026:	learn: 0.0932069	total: 1m 10s	remaining: 13s
2027:	learn: 0.0931961	total: 1m 10s	remaining: 13s
2028:	learn: 0.0931922	total: 1m 1

2166:	learn: 0.0918466	total: 1m 15s	remaining: 8.1s
2167:	learn: 0.0918439	total: 1m 15s	remaining: 8.06s
2168:	learn: 0.0918432	total: 1m 15s	remaining: 8.03s
2169:	learn: 0.0918302	total: 1m 15s	remaining: 7.99s
2170:	learn: 0.0918302	total: 1m 15s	remaining: 7.96s
2171:	learn: 0.0918245	total: 1m 15s	remaining: 7.92s
2172:	learn: 0.0918237	total: 1m 15s	remaining: 7.88s
2173:	learn: 0.0918077	total: 1m 15s	remaining: 7.85s
2174:	learn: 0.0918061	total: 1m 15s	remaining: 7.82s
2175:	learn: 0.0918050	total: 1m 15s	remaining: 7.78s
2176:	learn: 0.0917928	total: 1m 15s	remaining: 7.75s
2177:	learn: 0.0917888	total: 1m 15s	remaining: 7.71s
2178:	learn: 0.0917315	total: 1m 15s	remaining: 7.68s
2179:	learn: 0.0917203	total: 1m 15s	remaining: 7.64s
2180:	learn: 0.0917147	total: 1m 15s	remaining: 7.61s
2181:	learn: 0.0917127	total: 1m 15s	remaining: 7.57s
2182:	learn: 0.0917117	total: 1m 15s	remaining: 7.54s
2183:	learn: 0.0916974	total: 1m 15s	remaining: 7.5s
2184:	learn: 0.0916931	total: 

2322:	learn: 0.0900548	total: 1m 20s	remaining: 2.67s
2323:	learn: 0.0900376	total: 1m 20s	remaining: 2.63s
2324:	learn: 0.0900313	total: 1m 20s	remaining: 2.6s
2325:	learn: 0.0900306	total: 1m 20s	remaining: 2.56s
2326:	learn: 0.0900304	total: 1m 20s	remaining: 2.53s
2327:	learn: 0.0900298	total: 1m 20s	remaining: 2.49s
2328:	learn: 0.0900298	total: 1m 20s	remaining: 2.46s
2329:	learn: 0.0900274	total: 1m 20s	remaining: 2.42s
2330:	learn: 0.0899932	total: 1m 20s	remaining: 2.39s
2331:	learn: 0.0899925	total: 1m 20s	remaining: 2.35s
2332:	learn: 0.0899909	total: 1m 20s	remaining: 2.32s
2333:	learn: 0.0899897	total: 1m 20s	remaining: 2.28s
2334:	learn: 0.0899897	total: 1m 20s	remaining: 2.25s
2335:	learn: 0.0899897	total: 1m 20s	remaining: 2.21s
2336:	learn: 0.0899829	total: 1m 20s	remaining: 2.18s
2337:	learn: 0.0899775	total: 1m 20s	remaining: 2.14s
2338:	learn: 0.0899409	total: 1m 20s	remaining: 2.11s
2339:	learn: 0.0899393	total: 1m 20s	remaining: 2.08s
2340:	learn: 0.0899390	total:

,Model,Avg MSE,Avg Score
0,1.XGBRegressor,0.027334,0.883396
1,2.GradientBoostingRegressor,0.027123,0.882914
2,3.<catboost.core.CatBoostRegressor object at 0...,0.031854,0.867846
3,4.LGBMRegressor,0.029177,0.877155


```
xgbt1 = xgb.XGBRegressor(silent=1,max_leaf_nodes=255,seed=789,eta=0.03,scoring='neg_mean_squared_error',n_estimators=351, max_depth=5, max_features='sqrt', random_state=777, n_jobs=12)
xgbt2 = xgb.XGBRegressor(silent=1,max_leaf_nodes=255,seed=1000,eta=0.03,scoring='neg_mean_squared_error',n_estimators=361, max_depth=5, max_features='sqrt', random_state=999, n_jobs=12)
xgbt3 = xgb.XGBRegressor(silent=1,max_leaf_nodes=255,seed=1515,eta=0.03,scoring='neg_mean_squared_error',n_estimators=371, max_depth=5, max_features='sqrt', random_state=367, n_jobs=12)
# n_estimators=1000  max_depth=5  'sqrt'  GradientBoostingRegressor 最佳参数 ,learning_rate=0.08
gbdt1 = GradientBoostingRegressor(n_estimators=1060, max_depth=5, max_features='log2', random_state=789,learning_rate=0.08)
gbdt2 = GradientBoostingRegressor(n_estimators=1100, max_depth=5, max_features='log2', random_state=789,learning_rate=0.08)
gbdt3 = GradientBoostingRegressor(n_estimators=1090, max_depth=5, max_features='log2', random_state=789,learning_rate=0.08)
# n_estimators=700, max_features='auto', random_state=2, n_jobs=8,max_depth=10
forest1 = RandomForestRegressor(n_estimators=740, max_features='sqrt', random_state=7, n_jobs=12)
forest2 = RandomForestRegressor(n_estimators=730, max_features='sqrt', random_state=9, n_jobs=12)
forest3 = RandomForestRegressor(n_estimators=745, max_features='sqrt', random_state=11, n_jobs=12) 
# n_estimators 850 num_leaves=14,learning_rate=0.1 max_depth=5  seed
lgb1 = LGBMRegressor(n_estimators=840, max_depth=4, random_state=789, n_jobs=12,num_leaves=14,learning_rate=0.08,seed=666) 
lgb2 = LGBMRegressor(n_estimators=845, max_depth=4, random_state=798, n_jobs=12,num_leaves=13,learning_rate=0.1,seed=777)
lgb3 = LGBMRegressor(n_estimators=850, max_depth=4, random_state=777, n_jobs=12,num_leaves=10,learning_rate=0.1,seed=999)

	Model	Avg MSE	Avg Score
0	1.XGBRegressor	0.029245	0.869359
1	2.XGBRegressor	0.029234	0.869401
2	3.XGBRegressor	0.029219	0.869442
3	4.GradientBoostingRegressor	0.028651	0.869113
4	5.GradientBoostingRegressor	0.028638	0.869146
5	6.GradientBoostingRegressor	0.028641	0.869137
6	7.RandomForestRegressor	0.030268	0.865115
7	8.RandomForestRegressor	0.030265	0.865115
8	9.RandomForestRegressor	0.030372	0.864980
9	10.LGBMRegressor	0.029597	0.867302
10	11.LGBMRegressor	0.030036	0.866191
11	12.LGBMRegressor	0.030100	0.864830
```

```
xgbt1 = xgb.XGBRegressor(silent=1,max_leaf_nodes=255,seed=3000,eta=0.03,scoring='neg_mean_squared_error',n_estimators=351, max_depth=5, max_features='sqrt', random_state=2, n_jobs=12)
xgbt2 = xgb.XGBRegressor(silent=1,max_leaf_nodes=255,seed=3000,eta=0.03,scoring='neg_mean_squared_error',n_estimators=361, max_depth=5, max_features='sqrt', random_state=3, n_jobs=12)
xgbt3 = xgb.XGBRegressor(silent=1,max_leaf_nodes=255,seed=3000,eta=0.03,scoring='neg_mean_squared_error',n_estimators=371, max_depth=5, max_features='sqrt', random_state=4, n_jobs=12)
# n_estimators=1000  max_depth=5  'sqrt'  GradientBoostingRegressor 最佳参数 ,learning_rate=0.08
gbdt1 = GradientBoostingRegressor(n_estimators=1010, max_depth=5, max_features='log2', random_state=790,learning_rate=0.08)
gbdt2 = GradientBoostingRegressor(n_estimators=1000, max_depth=5, max_features='log2', random_state=791,learning_rate=0.08)
gbdt3 = GradientBoostingRegressor(n_estimators=1020, max_depth=5, max_features='log2', random_state=789,learning_rate=0.08)
# n_estimators=700, max_features='auto', random_state=2, n_jobs=8,max_depth=10
forest1 = RandomForestRegressor(n_estimators=720, max_features='sqrt', random_state=7, n_jobs=12)
forest2 = RandomForestRegressor(n_estimators=750, max_features='log2', random_state=9, n_jobs=12)
forest3 = RandomForestRegressor(n_estimators=800, max_features='sqrt', random_state=11, n_jobs=12) 

lgb1 = LGBMRegressor(n_estimators=800, max_depth=4, random_state=5, n_jobs=12) 
lgb2 = LGBMRegressor(n_estimators=850, max_depth=4, random_state=7, n_jobs=12)
lgb3 = LGBMRegressor(n_estimators=900, max_depth=4, random_state=9, n_jobs=12)

Model	Avg MSE	Avg Score
0	1.XGBRegressor	0.028357	0.871174
1	2.XGBRegressor	0.028332	0.871251
2	3.XGBRegressor	0.028333	0.871279
3	4.GradientBoostingRegressor	0.027985	0.871623
4	5.GradientBoostingRegressor	0.027273	0.872185
5	6.GradientBoostingRegressor	0.026847	0.874907
6	7.RandomForestRegressor	0.030499	0.865014
7	8.RandomForestRegressor	0.030333	0.864953
8	9.RandomForestRegressor	0.030495	0.864881
9	10.LGBMRegressor	0.029263	0.868385
10	11.LGBMRegressor	0.029263	0.868362
11	12.LGBMRegressor	0.029246	0.868430
```

```
xgbt1 = xgb.XGBRegressor(n_estimators=950, max_depth=3, max_features='sqrt', random_state=2, n_jobs=8)
xgbt2 = xgb.XGBRegressor(n_estimators=1000, max_depth=3, max_features='sqrt', random_state=3, n_jobs=8)
xgbt3 = xgb.XGBRegressor(n_estimators=1100, max_depth=3, max_features='sqrt', random_state=4, n_jobs=8)
# n_estimators=1000  max_depth=5  'sqrt'  GradientBoostingRegressor 最佳参数 ,learning_rate=0.08
gbdt1 = GradientBoostingRegressor(n_estimators=800, max_depth=4, max_features='log2', random_state=2,learning_rate=0.08)
gbdt2 = GradientBoostingRegressor(n_estimators=900, max_depth=4, max_features='log2', random_state=3,learning_rate=0.08)
gbdt3 = GradientBoostingRegressor(n_estimators=1000, max_depth=5, max_features='log2', random_state=4,learning_rate=0.08)
# n_estimators=700, max_features='auto', random_state=2, n_jobs=8,max_depth=10
forest1 = RandomForestRegressor(n_estimators=300, max_features='sqrt', random_state=2, n_jobs=8)
forest2 = RandomForestRegressor(n_estimators=400, max_features='log2', random_state=3, n_jobs=8)
forest3 = RandomForestRegressor(n_estimators=600, max_features='sqrt', random_state=4, n_jobs=8) 

lgb1 = LGBMRegressor(n_estimators=900, max_depth=5, random_state=2, n_jobs=8) 
lgb2 = LGBMRegressor(n_estimators=850, max_depth=4, random_state=3, n_jobs=8)
lgb3 = LGBMRegressor(n_estimators=720, max_depth=4, random_state=4, n_jobs=8)

Model	Avg MSE	Avg Score
0	1.XGBRegressor	0.029047	0.869022
1	2.XGBRegressor	0.029007	0.869151
2	3.XGBRegressor	0.028923	0.869379
3	4.GradientBoostingRegressor	0.028264	0.870069
4	5.GradientBoostingRegressor	0.028909	0.869134
5	6.GradientBoostingRegressor	0.027992	0.872356
6	7.RandomForestRegressor	0.030729	0.864344
7	8.RandomForestRegressor	0.030402	0.865025
8	9.RandomForestRegressor	0.030547	0.864977
9	10.LGBMRegressor	0.028749	0.868748
10	11.LGBMRegressor	0.029332	0.867618
11	12.LGBMRegressor	0.029250	0.867823
```

## randomForest 还可以调参再强化一下

In [ ]:
Model	Avg MSE	Avg Score
0	1.XGBRegressor	0.031286	0.862379
1	2.XGBRegressor	0.031232	0.862519
2	3.XGBRegressor	0.031166	0.862695
3	4.GradientBoostingRegressor	0.030385	0.863508
4	5.GradientBoostingRegressor	0.031893	0.859239
5	6.GradientBoostingRegressor	0.028836	0.868424
6	7.RandomForestRegressor	0.030334	0.865056
7	8.RandomForestRegressor	0.030304	0.865000
8	9.RandomForestRegressor	0.030405	0.864965
9	10.LGBMRegressor	0.029516	0.866988
10	11.LGBMRegressor	0.029343	0.867077
11	12.LGBMRegressor	0.029338	0.867135
Score: 0.8678230918561365 

In [ ]:
# train_data
# y = train_data['发电量']
# X = train_data.drop(['ID'], axis=1)
# sub_data = test_data.drop(['ID'], axis=1)
# print(type(clean_X_train))
# print(type(clean_X_test))
# print(type(clean_y_train))
# print(type(clean_y_test))
# print(type(clean_sub_data))

# print(type(X_train))
# print(type(X_test))
# print(type(y_train))
# print(type(y_test))
# print(type(sub_data))
# print(type(sm))
# print(type(polynm))

In [68]:
# categorical_features_indices = np.where(clean_X_train.dtypes != np.float)[0]

# train_pool = Pool(X_train, y_train,cat_features=categorical_features_indices)
# test_pool = Pool(X_test,cat_features=categorical_features_indices)
# model = CatBoostRegressor(iterations=None,
#                         learning_rate=0.05,
#                         l2_leaf_reg=0.15,
#                         border_count=150,
#                         max_depth=6,
#                         loss_function='RMSE',
#                         random_seed=3000,
#                         n_estimators=2400,
# #                         reg_lambda=2,
#                         leaf_estimation_method='Gradient',
#                         thread_count=12)
# model.fit(train_pool,plot=False,)
# cat_pred = model.predict(test_pool)
# mse = np.float(mean_squared_error(cat_pred,y_test))
# score = 1/(1+math.sqrt(mse))
# print(score)

CatboostError: Invalid type for cat_feature[0,29]=1.9947936 : cat_features must be integer or string, real number values and NaN values should be converted to string.

### Stacking

In [87]:
regrs = [
    xgbt1, gbdt1, cat1, lgb1,
#     xgbt2, gbdt2, forest2, lgb2,
#     xgbt3, gbdt3, forest3, lgb3
]

In [88]:
class Stacker(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models
    
    # X: 原始训练集, y: 原始训练集真实值, predict_data: 原始待预测数据
    def fit_predict(self, X, y, predict_data):
        X = np.array(X)
        y = np.array(y)
        T = np.array(predict_data)

        folds = list(KFold(n_splits=self.n_splits, shuffle=False, random_state=2018).split(X, y))
        
        # 以基学习器预测结果为特征的 stacker的训练数据 与 stacker预测数据
        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_predict = np.zeros((T.shape[0], len(self.base_models)))
        
        for i, regr in enumerate(self.base_models):
            print(i + 1, 'Base model:', str(regr).split('(')[0])
            S_predict_i = np.zeros((T.shape[0], self.n_splits))
            
            for j, (train_idx, test_idx) in enumerate(folds):
                # 将X分为训练集与测试集
                X_train, y_train, X_test, y_test = X[train_idx], y[train_idx], X[test_idx], y[test_idx]
                print ('Fit fold', (j+1), '...')
                regr.fit(X_train, y_train)
                y_pred = regr.predict(X_test)                
                S_train[test_idx, i] = y_pred
                S_predict_i[:, j] = regr.predict(T)
            
            S_predict[:, i] = S_predict_i.mean(axis=1)

        nmse_score = cross_val_score(self.stacker, S_train, y, cv=5, scoring='neg_mean_squared_error')
        print('CV MSE:', -nmse_score)
        print('Stacker AVG MSE:', -nmse_score.mean(), 'Stacker AVG Score:', np.mean(np.divide(1, 1 + np.sqrt(-nmse_score))))

        self.stacker.fit(S_train, y)
        res = self.stacker.predict(S_predict)
        return res, S_train, S_predict

In [89]:
# stacking_mode1 = Ridge(alpha=0.008, copy_X=True, fit_intercept=False, solver='auto', random_state=2)
stacking_model = SVR(C=100, gamma=0.01, epsilon=0.01)
stacker = Stacker(5, stacking_model, regrs)
pred_stack, S_train_data, S_predict_data = stacker.fit_predict(all_X_train, all_y_train, sub_data)

1 Base model: XGBRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
2 Base model: GradientBoostingRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
3 Base model: <catboost.core.CatBoostRegressor object at 0x0000019019874CF8>
Fit fold 1 ...
0:	learn: 6.3914979	total: 34.1ms	remaining: 1m 21s
1:	learn: 6.0798649	total: 69.8ms	remaining: 1m 23s
2:	learn: 5.7817082	total: 113ms	remaining: 1m 30s
3:	learn: 5.4971974	total: 145ms	remaining: 1m 26s
4:	learn: 5.2263594	total: 178ms	remaining: 1m 25s
5:	learn: 4.9694030	total: 208ms	remaining: 1m 23s
6:	learn: 4.7259656	total: 242ms	remaining: 1m 22s
7:	learn: 4.4965345	total: 274ms	remaining: 1m 21s
8:	learn: 4.2778369	total: 318ms	remaining: 1m 24s
9:	learn: 4.0687776	total: 354ms	remaining: 1m 24s
10:	learn: 3.8701846	total: 391ms	remaining: 1m 24s
11:	learn: 3.6832214	total: 432ms	remaining: 1m 25s
12:	learn: 3.5035596	total: 468ms	remaining: 1m 25s
13:	learn: 3.3341175	to

153:	learn: 0.1880217	total: 5.32s	remaining: 1m 17s
154:	learn: 0.1877929	total: 5.36s	remaining: 1m 17s
155:	learn: 0.1876475	total: 5.39s	remaining: 1m 17s
156:	learn: 0.1874389	total: 5.42s	remaining: 1m 17s
157:	learn: 0.1871161	total: 5.45s	remaining: 1m 17s
158:	learn: 0.1869419	total: 5.49s	remaining: 1m 17s
159:	learn: 0.1866178	total: 5.52s	remaining: 1m 17s
160:	learn: 0.1863119	total: 5.55s	remaining: 1m 17s
161:	learn: 0.1861887	total: 5.6s	remaining: 1m 17s
162:	learn: 0.1859137	total: 5.63s	remaining: 1m 17s
163:	learn: 0.1857136	total: 5.67s	remaining: 1m 17s
164:	learn: 0.1855013	total: 5.71s	remaining: 1m 17s
165:	learn: 0.1851783	total: 5.74s	remaining: 1m 17s
166:	learn: 0.1849231	total: 5.78s	remaining: 1m 17s
167:	learn: 0.1847460	total: 5.81s	remaining: 1m 17s
168:	learn: 0.1845932	total: 5.85s	remaining: 1m 17s
169:	learn: 0.1843675	total: 5.88s	remaining: 1m 17s
170:	learn: 0.1841516	total: 5.91s	remaining: 1m 17s
171:	learn: 0.1840257	total: 5.95s	remaining: 1

310:	learn: 0.1660295	total: 10.6s	remaining: 1m 11s
311:	learn: 0.1660059	total: 10.7s	remaining: 1m 11s
312:	learn: 0.1659951	total: 10.7s	remaining: 1m 11s
313:	learn: 0.1658935	total: 10.8s	remaining: 1m 11s
314:	learn: 0.1657943	total: 10.8s	remaining: 1m 11s
315:	learn: 0.1657757	total: 10.8s	remaining: 1m 11s
316:	learn: 0.1657048	total: 10.9s	remaining: 1m 11s
317:	learn: 0.1656772	total: 10.9s	remaining: 1m 11s
318:	learn: 0.1656372	total: 10.9s	remaining: 1m 11s
319:	learn: 0.1656008	total: 11s	remaining: 1m 11s
320:	learn: 0.1655035	total: 11s	remaining: 1m 11s
321:	learn: 0.1654653	total: 11s	remaining: 1m 11s
322:	learn: 0.1654353	total: 11.1s	remaining: 1m 11s
323:	learn: 0.1654085	total: 11.1s	remaining: 1m 11s
324:	learn: 0.1652876	total: 11.1s	remaining: 1m 11s
325:	learn: 0.1652676	total: 11.2s	remaining: 1m 11s
326:	learn: 0.1652429	total: 11.2s	remaining: 1m 10s
327:	learn: 0.1646882	total: 11.2s	remaining: 1m 10s
328:	learn: 0.1646388	total: 11.3s	remaining: 1m 10s

468:	learn: 0.1558328	total: 16s	remaining: 1m 5s
469:	learn: 0.1558244	total: 16s	remaining: 1m 5s
470:	learn: 0.1558059	total: 16.1s	remaining: 1m 5s
471:	learn: 0.1554095	total: 16.1s	remaining: 1m 5s
472:	learn: 0.1552947	total: 16.1s	remaining: 1m 5s
473:	learn: 0.1552707	total: 16.1s	remaining: 1m 5s
474:	learn: 0.1552241	total: 16.2s	remaining: 1m 5s
475:	learn: 0.1551856	total: 16.2s	remaining: 1m 5s
476:	learn: 0.1550515	total: 16.2s	remaining: 1m 5s
477:	learn: 0.1550450	total: 16.3s	remaining: 1m 5s
478:	learn: 0.1550234	total: 16.3s	remaining: 1m 5s
479:	learn: 0.1550007	total: 16.3s	remaining: 1m 5s
480:	learn: 0.1549289	total: 16.4s	remaining: 1m 5s
481:	learn: 0.1548859	total: 16.4s	remaining: 1m 5s
482:	learn: 0.1548758	total: 16.4s	remaining: 1m 5s
483:	learn: 0.1547856	total: 16.5s	remaining: 1m 5s
484:	learn: 0.1547667	total: 16.5s	remaining: 1m 5s
485:	learn: 0.1545926	total: 16.5s	remaining: 1m 5s
486:	learn: 0.1545429	total: 16.6s	remaining: 1m 5s
487:	learn: 0.15

629:	learn: 0.1489506	total: 21.3s	remaining: 59.9s
630:	learn: 0.1489487	total: 21.3s	remaining: 59.8s
631:	learn: 0.1488288	total: 21.4s	remaining: 59.8s
632:	learn: 0.1488287	total: 21.4s	remaining: 59.7s
633:	learn: 0.1487370	total: 21.4s	remaining: 59.7s
634:	learn: 0.1486916	total: 21.5s	remaining: 59.7s
635:	learn: 0.1486909	total: 21.5s	remaining: 59.6s
636:	learn: 0.1486130	total: 21.5s	remaining: 59.6s
637:	learn: 0.1484590	total: 21.6s	remaining: 59.6s
638:	learn: 0.1484590	total: 21.6s	remaining: 59.5s
639:	learn: 0.1484587	total: 21.6s	remaining: 59.4s
640:	learn: 0.1484422	total: 21.6s	remaining: 59.4s
641:	learn: 0.1484373	total: 21.7s	remaining: 59.4s
642:	learn: 0.1484256	total: 21.7s	remaining: 59.3s
643:	learn: 0.1482509	total: 21.7s	remaining: 59.3s
644:	learn: 0.1482413	total: 21.8s	remaining: 59.3s
645:	learn: 0.1482124	total: 21.8s	remaining: 59.2s
646:	learn: 0.1482080	total: 21.9s	remaining: 59.2s
647:	learn: 0.1481652	total: 21.9s	remaining: 59.2s
648:	learn: 

...

1712:	learn: 0.1011528	total: 57.9s	remaining: 23.2s
1713:	learn: 0.1011526	total: 57.9s	remaining: 23.2s
1714:	learn: 0.1010873	total: 57.9s	remaining: 23.1s
1715:	learn: 0.1010794	total: 58s	remaining: 23.1s
1716:	learn: 0.1010083	total: 58s	remaining: 23.1s
1717:	learn: 0.1009821	total: 58s	remaining: 23s
1718:	learn: 0.1009708	total: 58s	remaining: 23s
1719:	learn: 0.1009676	total: 58.1s	remaining: 23s
1720:	learn: 0.1009038	total: 58.1s	remaining: 22.9s
1721:	learn: 0.1008675	total: 58.1s	remaining: 22.9s
1722:	learn: 0.1008620	total: 58.2s	remaining: 22.9s
1723:	learn: 0.1008600	total: 58.2s	remaining: 22.8s
1724:	learn: 0.1008566	total: 58.2s	remaining: 22.8s
1725:	learn: 0.1008556	total: 58.3s	remaining: 22.8s
1726:	learn: 0.1008336	total: 58.3s	remaining: 22.7s
1727:	learn: 0.1008333	total: 58.4s	remaining: 22.7s
1728:	learn: 0.1007923	total: 58.4s	remaining: 22.7s
1729:	learn: 0.1007538	total: 58.4s	remaining: 22.6s
1730:	learn: 0.1007534	total: 58.5s	remaining: 22.6s
1731:	l

1870:	learn: 0.0978858	total: 1m 3s	remaining: 17.9s
1871:	learn: 0.0978842	total: 1m 3s	remaining: 17.9s
1872:	learn: 0.0978790	total: 1m 3s	remaining: 17.9s
1873:	learn: 0.0978362	total: 1m 3s	remaining: 17.8s
1874:	learn: 0.0978285	total: 1m 3s	remaining: 17.8s
1875:	learn: 0.0978144	total: 1m 3s	remaining: 17.8s
1876:	learn: 0.0978064	total: 1m 3s	remaining: 17.7s
1877:	learn: 0.0978043	total: 1m 3s	remaining: 17.7s
1878:	learn: 0.0977856	total: 1m 3s	remaining: 17.7s
1879:	learn: 0.0977764	total: 1m 3s	remaining: 17.6s
1880:	learn: 0.0977422	total: 1m 3s	remaining: 17.6s
1881:	learn: 0.0977323	total: 1m 3s	remaining: 17.6s
1882:	learn: 0.0977082	total: 1m 3s	remaining: 17.5s
1883:	learn: 0.0977074	total: 1m 3s	remaining: 17.5s
1884:	learn: 0.0977050	total: 1m 3s	remaining: 17.5s
1885:	learn: 0.0976929	total: 1m 3s	remaining: 17.4s
1886:	learn: 0.0976583	total: 1m 3s	remaining: 17.4s
1887:	learn: 0.0976561	total: 1m 4s	remaining: 17.4s
1888:	learn: 0.0976521	total: 1m 4s	remaining:

2026:	learn: 0.0952995	total: 1m 8s	remaining: 12.6s
2027:	learn: 0.0952981	total: 1m 8s	remaining: 12.6s
2028:	learn: 0.0950191	total: 1m 8s	remaining: 12.6s
2029:	learn: 0.0949850	total: 1m 8s	remaining: 12.5s
2030:	learn: 0.0949778	total: 1m 8s	remaining: 12.5s
2031:	learn: 0.0949686	total: 1m 8s	remaining: 12.5s
2032:	learn: 0.0949658	total: 1m 8s	remaining: 12.4s
2033:	learn: 0.0949235	total: 1m 8s	remaining: 12.4s
2034:	learn: 0.0949217	total: 1m 8s	remaining: 12.4s
2035:	learn: 0.0949065	total: 1m 8s	remaining: 12.3s
2036:	learn: 0.0949019	total: 1m 9s	remaining: 12.3s
2037:	learn: 0.0949003	total: 1m 9s	remaining: 12.3s
2038:	learn: 0.0948973	total: 1m 9s	remaining: 12.2s
2039:	learn: 0.0948612	total: 1m 9s	remaining: 12.2s
2040:	learn: 0.0948603	total: 1m 9s	remaining: 12.2s
2041:	learn: 0.0948597	total: 1m 9s	remaining: 12.1s
2042:	learn: 0.0948255	total: 1m 9s	remaining: 12.1s
2043:	learn: 0.0948225	total: 1m 9s	remaining: 12.1s
2044:	learn: 0.0948191	total: 1m 9s	remaining:

2183:	learn: 0.0927845	total: 1m 13s	remaining: 7.29s
2184:	learn: 0.0927255	total: 1m 13s	remaining: 7.26s
2185:	learn: 0.0927198	total: 1m 13s	remaining: 7.22s
2186:	learn: 0.0927036	total: 1m 13s	remaining: 7.19s
2187:	learn: 0.0926837	total: 1m 13s	remaining: 7.16s
2188:	learn: 0.0926570	total: 1m 13s	remaining: 7.12s
2189:	learn: 0.0926211	total: 1m 13s	remaining: 7.09s
2190:	learn: 0.0926179	total: 1m 13s	remaining: 7.05s
2191:	learn: 0.0926069	total: 1m 13s	remaining: 7.02s
2192:	learn: 0.0926044	total: 1m 14s	remaining: 6.99s
2193:	learn: 0.0926022	total: 1m 14s	remaining: 6.95s
2194:	learn: 0.0925968	total: 1m 14s	remaining: 6.92s
2195:	learn: 0.0925906	total: 1m 14s	remaining: 6.88s
2196:	learn: 0.0925550	total: 1m 14s	remaining: 6.85s
2197:	learn: 0.0925543	total: 1m 14s	remaining: 6.82s
2198:	learn: 0.0925507	total: 1m 14s	remaining: 6.78s
2199:	learn: 0.0925481	total: 1m 14s	remaining: 6.75s
2200:	learn: 0.0925047	total: 1m 14s	remaining: 6.72s
2201:	learn: 0.0924962	total

2341:	learn: 0.0901690	total: 1m 18s	remaining: 1.95s
2342:	learn: 0.0901672	total: 1m 18s	remaining: 1.92s
2343:	learn: 0.0901623	total: 1m 18s	remaining: 1.89s
2344:	learn: 0.0901491	total: 1m 19s	remaining: 1.85s
2345:	learn: 0.0901368	total: 1m 19s	remaining: 1.82s
2346:	learn: 0.0901353	total: 1m 19s	remaining: 1.78s
2347:	learn: 0.0900406	total: 1m 19s	remaining: 1.75s
2348:	learn: 0.0900366	total: 1m 19s	remaining: 1.72s
2349:	learn: 0.0900361	total: 1m 19s	remaining: 1.68s
2350:	learn: 0.0900313	total: 1m 19s	remaining: 1.65s
2351:	learn: 0.0900292	total: 1m 19s	remaining: 1.62s
2352:	learn: 0.0900272	total: 1m 19s	remaining: 1.58s
2353:	learn: 0.0900209	total: 1m 19s	remaining: 1.55s
2354:	learn: 0.0900162	total: 1m 19s	remaining: 1.52s
2355:	learn: 0.0900150	total: 1m 19s	remaining: 1.48s
2356:	learn: 0.0899968	total: 1m 19s	remaining: 1.45s
2357:	learn: 0.0899953	total: 1m 19s	remaining: 1.42s
2358:	learn: 0.0899273	total: 1m 19s	remaining: 1.38s
2359:	learn: 0.0899077	total

### 日期时间 2018年8月10日8点45分
```
CV MSE: [0.00864028 0.01972439 0.00933717 0.08534433 0.01271464]
Stacker AVG MSE: 0.027152161227452933 Stacker AVG Score: 0.8752536047385997
```

In [ ]:
CV MSE: [0.00834157 0.01945687 0.0095695  0.08376848 0.01302208]
Stacker AVG MSE: 0.02683170221309707 Stacker AVG Score: 0.8755806306487525

In [90]:
stacking_model2 = SVR(C=100, gamma=0.01, epsilon=0.01)
stacker2 = Stacker(5, stacking_model2, regrs)
pred_clean_stack, S_clean_train_data, S_clean_predict_data = stacker2.fit_predict(clean_X, clean_y, clean_sub_data)

1 Base model: XGBRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
2 Base model: GradientBoostingRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
3 Base model: <catboost.core.CatBoostRegressor object at 0x0000019019874CF8>
Fit fold 1 ...
0:	learn: 6.4766881	total: 31.5ms	remaining: 1m 15s
1:	learn: 6.1597872	total: 64.5ms	remaining: 1m 17s
2:	learn: 5.8616449	total: 97.6ms	remaining: 1m 18s
3:	learn: 5.5751661	total: 128ms	remaining: 1m 16s
4:	learn: 5.3034152	total: 159ms	remaining: 1m 16s
5:	learn: 5.0443333	total: 190ms	remaining: 1m 15s
6:	learn: 4.7969357	total: 222ms	remaining: 1m 15s
7:	learn: 4.5614398	total: 258ms	remaining: 1m 17s
8:	learn: 4.3380470	total: 292ms	remaining: 1m 17s
9:	learn: 4.1262332	total: 324ms	remaining: 1m 17s
10:	learn: 3.9263452	total: 356ms	remaining: 1m 17s
11:	learn: 3.7354110	total: 387ms	remaining: 1m 17s
12:	learn: 3.5525842	total: 419ms	remaining: 1m 17s
13:	learn: 3.3793271	t

152:	learn: 0.1869035	total: 5.11s	remaining: 1m 15s
153:	learn: 0.1867849	total: 5.15s	remaining: 1m 15s
154:	learn: 0.1867736	total: 5.18s	remaining: 1m 15s
155:	learn: 0.1864993	total: 5.22s	remaining: 1m 15s
156:	learn: 0.1862427	total: 5.25s	remaining: 1m 15s
157:	learn: 0.1858982	total: 5.29s	remaining: 1m 15s
158:	learn: 0.1856947	total: 5.33s	remaining: 1m 15s
159:	learn: 0.1854696	total: 5.36s	remaining: 1m 15s
160:	learn: 0.1851222	total: 5.39s	remaining: 1m 14s
161:	learn: 0.1848009	total: 5.42s	remaining: 1m 14s
162:	learn: 0.1846918	total: 5.45s	remaining: 1m 14s
163:	learn: 0.1846342	total: 5.49s	remaining: 1m 14s
164:	learn: 0.1844169	total: 5.52s	remaining: 1m 14s
165:	learn: 0.1843121	total: 5.56s	remaining: 1m 14s
166:	learn: 0.1842158	total: 5.59s	remaining: 1m 14s
167:	learn: 0.1838234	total: 5.63s	remaining: 1m 14s
168:	learn: 0.1834859	total: 5.67s	remaining: 1m 14s
169:	learn: 0.1831844	total: 5.71s	remaining: 1m 14s
170:	learn: 0.1831412	total: 5.74s	remaining: 

308:	learn: 0.1514024	total: 10.2s	remaining: 1m 9s
309:	learn: 0.1511400	total: 10.3s	remaining: 1m 9s
310:	learn: 0.1510193	total: 10.3s	remaining: 1m 9s
311:	learn: 0.1507175	total: 10.3s	remaining: 1m 9s
312:	learn: 0.1501731	total: 10.3s	remaining: 1m 9s
313:	learn: 0.1499671	total: 10.4s	remaining: 1m 8s
314:	learn: 0.1499282	total: 10.4s	remaining: 1m 8s
315:	learn: 0.1497654	total: 10.4s	remaining: 1m 8s
316:	learn: 0.1497143	total: 10.5s	remaining: 1m 8s
317:	learn: 0.1496160	total: 10.5s	remaining: 1m 8s
318:	learn: 0.1496075	total: 10.5s	remaining: 1m 8s
319:	learn: 0.1493955	total: 10.6s	remaining: 1m 8s
320:	learn: 0.1492759	total: 10.6s	remaining: 1m 8s
321:	learn: 0.1492712	total: 10.6s	remaining: 1m 8s
322:	learn: 0.1490727	total: 10.7s	remaining: 1m 8s
323:	learn: 0.1489723	total: 10.7s	remaining: 1m 8s
324:	learn: 0.1487825	total: 10.7s	remaining: 1m 8s
325:	learn: 0.1486528	total: 10.8s	remaining: 1m 8s
326:	learn: 0.1485161	total: 10.8s	remaining: 1m 8s
327:	learn: 

470:	learn: 0.1275390	total: 15.4s	remaining: 1m 2s
471:	learn: 0.1275143	total: 15.4s	remaining: 1m 2s
472:	learn: 0.1264404	total: 15.4s	remaining: 1m 2s
473:	learn: 0.1264264	total: 15.5s	remaining: 1m 2s
474:	learn: 0.1264043	total: 15.5s	remaining: 1m 2s
475:	learn: 0.1263723	total: 15.5s	remaining: 1m 2s
476:	learn: 0.1259543	total: 15.6s	remaining: 1m 2s
477:	learn: 0.1258956	total: 15.6s	remaining: 1m 2s
478:	learn: 0.1258593	total: 15.6s	remaining: 1m 2s
479:	learn: 0.1257471	total: 15.7s	remaining: 1m 2s
480:	learn: 0.1257426	total: 15.7s	remaining: 1m 2s
481:	learn: 0.1256462	total: 15.7s	remaining: 1m 2s
482:	learn: 0.1253736	total: 15.8s	remaining: 1m 2s
483:	learn: 0.1253427	total: 15.8s	remaining: 1m 2s
484:	learn: 0.1253422	total: 15.8s	remaining: 1m 2s
485:	learn: 0.1253327	total: 15.8s	remaining: 1m 2s
486:	learn: 0.1252922	total: 15.9s	remaining: 1m 2s
487:	learn: 0.1252922	total: 15.9s	remaining: 1m 2s
488:	learn: 0.1251884	total: 15.9s	remaining: 1m 2s
489:	learn: 

637:	learn: 0.1163315	total: 20.5s	remaining: 56.7s
638:	learn: 0.1163274	total: 20.6s	remaining: 56.7s
639:	learn: 0.1162723	total: 20.6s	remaining: 56.7s
640:	learn: 0.1155240	total: 20.6s	remaining: 56.6s
641:	learn: 0.1155126	total: 20.7s	remaining: 56.6s
642:	learn: 0.1155124	total: 20.7s	remaining: 56.5s
643:	learn: 0.1154154	total: 20.7s	remaining: 56.5s
644:	learn: 0.1152631	total: 20.8s	remaining: 56.5s
645:	learn: 0.1152588	total: 20.8s	remaining: 56.4s
646:	learn: 0.1148976	total: 20.8s	remaining: 56.4s
647:	learn: 0.1148673	total: 20.8s	remaining: 56.4s
648:	learn: 0.1147086	total: 20.9s	remaining: 56.3s
649:	learn: 0.1146971	total: 20.9s	remaining: 56.3s
650:	learn: 0.1146921	total: 21s	remaining: 56.3s
651:	learn: 0.1146383	total: 21s	remaining: 56.3s
652:	learn: 0.1146191	total: 21s	remaining: 56.2s
653:	learn: 0.1146191	total: 21s	remaining: 56.1s
654:	learn: 0.1146191	total: 21s	remaining: 56.1s
655:	learn: 0.1145970	total: 21.1s	remaining: 56s
656:	learn: 0.1145275	to

...

1729:	learn: 0.0901137	total: 55.2s	remaining: 21.4s
1730:	learn: 0.0901036	total: 55.2s	remaining: 21.3s
1731:	learn: 0.0900931	total: 55.2s	remaining: 21.3s
1732:	learn: 0.0900731	total: 55.3s	remaining: 21.3s
1733:	learn: 0.0900389	total: 55.3s	remaining: 21.2s
1734:	learn: 0.0900356	total: 55.3s	remaining: 21.2s
1735:	learn: 0.0900201	total: 55.4s	remaining: 21.2s
1736:	learn: 0.0900173	total: 55.4s	remaining: 21.1s
1737:	learn: 0.0899984	total: 55.4s	remaining: 21.1s
1738:	learn: 0.0899983	total: 55.4s	remaining: 21.1s
1739:	learn: 0.0899835	total: 55.5s	remaining: 21s
1740:	learn: 0.0899749	total: 55.5s	remaining: 21s
1741:	learn: 0.0899561	total: 55.5s	remaining: 21s
1742:	learn: 0.0899504	total: 55.6s	remaining: 20.9s
1743:	learn: 0.0899257	total: 55.6s	remaining: 20.9s
1744:	learn: 0.0898918	total: 55.6s	remaining: 20.9s
1745:	learn: 0.0898883	total: 55.7s	remaining: 20.8s
1746:	learn: 0.0898850	total: 55.7s	remaining: 20.8s
1747:	learn: 0.0898755	total: 55.7s	remaining: 20.8s

1889:	learn: 0.0873996	total: 1m	remaining: 16.3s
1890:	learn: 0.0873986	total: 1m	remaining: 16.2s
1891:	learn: 0.0873839	total: 1m	remaining: 16.2s
1892:	learn: 0.0873839	total: 1m	remaining: 16.2s
1893:	learn: 0.0873683	total: 1m	remaining: 16.1s
1894:	learn: 0.0873626	total: 1m	remaining: 16.1s
1895:	learn: 0.0873550	total: 1m	remaining: 16.1s
1896:	learn: 0.0873532	total: 1m	remaining: 16s
1897:	learn: 0.0873452	total: 1m	remaining: 16s
1898:	learn: 0.0873171	total: 1m	remaining: 16s
1899:	learn: 0.0873122	total: 1m	remaining: 15.9s
1900:	learn: 0.0873003	total: 1m	remaining: 15.9s
1901:	learn: 0.0872978	total: 1m	remaining: 15.9s
1902:	learn: 0.0872767	total: 1m	remaining: 15.8s
1903:	learn: 0.0872582	total: 1m	remaining: 15.8s
1904:	learn: 0.0872582	total: 1m	remaining: 15.8s
1905:	learn: 0.0872485	total: 1m	remaining: 15.7s
1906:	learn: 0.0872419	total: 1m	remaining: 15.7s
1907:	learn: 0.0872310	total: 1m	remaining: 15.7s
1908:	learn: 0.0872310	total: 1m	remaining: 15.6s
1909:	

2048:	learn: 0.0852153	total: 1m 5s	remaining: 11.2s
2049:	learn: 0.0852019	total: 1m 5s	remaining: 11.1s
2050:	learn: 0.0851983	total: 1m 5s	remaining: 11.1s
2051:	learn: 0.0851850	total: 1m 5s	remaining: 11.1s
2052:	learn: 0.0851656	total: 1m 5s	remaining: 11s
2053:	learn: 0.0851643	total: 1m 5s	remaining: 11s
2054:	learn: 0.0851162	total: 1m 5s	remaining: 11s
2055:	learn: 0.0850809	total: 1m 5s	remaining: 10.9s
2056:	learn: 0.0850721	total: 1m 5s	remaining: 10.9s
2057:	learn: 0.0850694	total: 1m 5s	remaining: 10.9s
2058:	learn: 0.0850461	total: 1m 5s	remaining: 10.9s
2059:	learn: 0.0850276	total: 1m 5s	remaining: 10.8s
2060:	learn: 0.0850163	total: 1m 5s	remaining: 10.8s
2061:	learn: 0.0850159	total: 1m 5s	remaining: 10.8s
2062:	learn: 0.0850069	total: 1m 5s	remaining: 10.7s
2063:	learn: 0.0850034	total: 1m 5s	remaining: 10.7s
2064:	learn: 0.0849966	total: 1m 5s	remaining: 10.7s
2065:	learn: 0.0849827	total: 1m 5s	remaining: 10.6s
2066:	learn: 0.0849641	total: 1m 5s	remaining: 10.6s

2205:	learn: 0.0831142	total: 1m 10s	remaining: 6.17s
2206:	learn: 0.0831136	total: 1m 10s	remaining: 6.14s
2207:	learn: 0.0831136	total: 1m 10s	remaining: 6.1s
2208:	learn: 0.0830384	total: 1m 10s	remaining: 6.07s
2209:	learn: 0.0830367	total: 1m 10s	remaining: 6.04s
2210:	learn: 0.0830107	total: 1m 10s	remaining: 6.01s
2211:	learn: 0.0829446	total: 1m 10s	remaining: 5.98s
2212:	learn: 0.0829298	total: 1m 10s	remaining: 5.95s
2213:	learn: 0.0829152	total: 1m 10s	remaining: 5.91s
2214:	learn: 0.0829090	total: 1m 10s	remaining: 5.88s
2215:	learn: 0.0828697	total: 1m 10s	remaining: 5.85s
2216:	learn: 0.0828621	total: 1m 10s	remaining: 5.82s
2217:	learn: 0.0828528	total: 1m 10s	remaining: 5.79s
2218:	learn: 0.0828495	total: 1m 10s	remaining: 5.75s
2219:	learn: 0.0828388	total: 1m 10s	remaining: 5.72s
2220:	learn: 0.0828345	total: 1m 10s	remaining: 5.69s
2221:	learn: 0.0828345	total: 1m 10s	remaining: 5.66s
2222:	learn: 0.0828345	total: 1m 10s	remaining: 5.62s
2223:	learn: 0.0828223	total:

2360:	learn: 0.0809551	total: 1m 14s	remaining: 1.24s
2361:	learn: 0.0809532	total: 1m 15s	remaining: 1.21s
2362:	learn: 0.0809245	total: 1m 15s	remaining: 1.18s
2363:	learn: 0.0809210	total: 1m 15s	remaining: 1.14s
2364:	learn: 0.0809017	total: 1m 15s	remaining: 1.11s
2365:	learn: 0.0809008	total: 1m 15s	remaining: 1.08s
2366:	learn: 0.0808920	total: 1m 15s	remaining: 1.05s
2367:	learn: 0.0808827	total: 1m 15s	remaining: 1.02s
2368:	learn: 0.0808821	total: 1m 15s	remaining: 985ms
2369:	learn: 0.0808708	total: 1m 15s	remaining: 953ms
2370:	learn: 0.0808318	total: 1m 15s	remaining: 921ms
2371:	learn: 0.0808301	total: 1m 15s	remaining: 889ms
2372:	learn: 0.0808276	total: 1m 15s	remaining: 858ms
2373:	learn: 0.0808255	total: 1m 15s	remaining: 826ms
2374:	learn: 0.0808077	total: 1m 15s	remaining: 794ms
2375:	learn: 0.0807986	total: 1m 15s	remaining: 762ms
2376:	learn: 0.0807755	total: 1m 15s	remaining: 731ms
2377:	learn: 0.0807743	total: 1m 15s	remaining: 699ms
2378:	learn: 0.0807738	total

### 日期时间 2018年8月10日8点45分
```
CV MSE: [0.00986441 0.01838237 0.05783231 0.01100778 0.009911  ]
Stacker AVG MSE: 0.021399573460694675 Stacker AVG Score: 0.8821802017381792
```

In [ ]:
CV MSE: [0.01659384 0.00626714 0.01261921 0.05345451 0.01616147]
Stacker AVG MSE: 0.021019233812974467 Stacker AVG Score: 0.8821919685074249

In [ ]:
CV MSE: [0.00902831 0.01733493 0.05746076 0.01078939 0.00961003]
Stacker AVG MSE: 0.020844682710295333 Stacker AVG Score: 0.8840294650706134

### Output

In [91]:
df_result['score'] = pred_stack

In [92]:
index = df_result[df_result['ID'].isin(special_missing_ID)].index
df_result.loc[index, 'score'] = 0.379993053

In [93]:
c_index = df_result[df_result['ID'].isin(cleaned_sub_data_ID)].index
df_result.loc[c_index, 'score'] = pred_clean_stack

In [94]:
df_result.to_csv('8_10_time_15.13.version4.0.csv', index=False, header=False)